In [2]:
#import statements

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import os

import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

from sklearn.metrics import roc_curve, auc


# the size of the images in the PCAM dataset
IMAGE_SIZE = 96

In [3]:
#get data from patchCAMELYON dataset

def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     TRAIN_PATH = os.path.join(base_dir, 'train+val', 'train')
     VALID_PATH = os.path.join(base_dir, 'train+val', 'valid')

     RESCALING_FACTOR = 1./255
     
     # instantiate data generators
     datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

     train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(VALID_PATH,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary',
                                             shuffle=False)
     
     return train_gen, val_gen

train_gen, val_gen = get_pcam_generators('C:/Users/20212150/Documents/Year 3/Q3/Project AI mia')
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [4]:
#define autoencoder
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape), activation='sigmoid'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


shape = (96,96)
latent_dim = 64
autoencoder = Autoencoder(latent_dim, shape)

In [12]:
print((train_gen[3]))

(array([[[[0.5058824 , 0.29803923, 0.47058827],
         [0.7176471 , 0.5019608 , 0.654902  ],
         [0.7568628 , 0.5411765 , 0.6666667 ],
         ...,
         [0.77647066, 0.5294118 , 0.6039216 ],
         [0.74509805, 0.48235297, 0.54901963],
         [0.81568635, 0.5372549 , 0.6       ]],

        [[0.8941177 , 0.6745098 , 0.82745105],
         [0.7411765 , 0.52156866, 0.65882355],
         [0.73333335, 0.5137255 , 0.627451  ],
         ...,
         [0.83921576, 0.6117647 , 0.70980394],
         [0.882353  , 0.6392157 , 0.7372549 ],
         [0.8117648 , 0.56078434, 0.6627451 ]],

        [[0.7725491 , 0.5411765 , 0.65882355],
         [0.70980394, 0.4784314 , 0.5882353 ],
         [0.8588236 , 0.62352943, 0.7254902 ],
         ...,
         [0.38823533, 0.19215688, 0.32941177],
         [0.5882353 , 0.38431376, 0.5254902 ],
         [0.6862745 , 0.47450984, 0.6156863 ]],

        ...,

        [[0.30980393, 0.14117648, 0.30980393],
         [0.3137255 , 0.14509805, 0.32156864

In [14]:
#train autoencoder
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(train_gen, train_gen,
                epochs=10,
                shuffle=True,
                validation_data=(val_gen, val_gen))

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.